**Install [vLLM](https://docs.vllm.ai/en/latest/getting_started/quickstart.html) package**

In [ ]:
!pip install vllm lm-format-enforcer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 M

**Run vLLM server background using nohup. It can take a while for the first-time running, as it would download a model from Huggingface**

In [ ]:
!nohup vllm serve Qwen/Qwen2.5-1.5B-Instruct &

nohup: appending output to 'nohup.out'


In [ ]:
# Test connection:
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"Qwen/Qwen2.5-1.5B-Instruct","object":"model","created":1737989894,"owned_by":"vllm","root":"Qwen/Qwen2.5-1.5B-Instruct","parent":null,"max_model_len":32768,"permission":[{"id":"modelperm-138880724f224afc802f709bb5fc4ef8","object":"model_permission","created":1737989894,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

**Install [ngrok](https://ngrok.com/) for explosing the local vLLM on a colab server to the Internet**

In [ ]:
!curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
	| sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
	&& echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
	| sudo tee /etc/apt/sources.list.d/ngrok.list \
	&& sudo apt update \
	&& sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,285 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.7 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Packages [7,389 B]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [

**Follow instruction on [Ngrok dashboard](https://dashboard.ngrok.com/get-started/setup/linux) to tunel a local server to the Internet**

In [ ]:
# Replace your authtoken
!ngrok config add-authtoken your_authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Tunel local server to a public URL on ngork
!ngrok http http://localhost:8000 --log=stdout > ngrok.log &

In [ ]:
!curl https://a1e4-193-61-202-14.ngrok-free.app/v1/models

{"object":"list","data":[{"id":"Qwen/Qwen2.5-1.5B-Instruct","object":"model","created":1737989912,"owned_by":"vllm","root":"Qwen/Qwen2.5-1.5B-Instruct","parent":null,"max_model_len":32768,"permission":[{"id":"modelperm-dc43f971056b4738a6e7531715795990","object":"model_permission","created":1737989912,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [ ]:
!curl https://a1e4-193-61-202-14.ngrok-free.app/v1/models

{"object":"list","data":[{"id":"deepseek-ai/DeepSeek-R1-Distill-Qwen-7B","object":"model","created":1738330774,"owned_by":"vllm","root":"deepseek-ai/DeepSeek-R1-Distill-Qwen-7B","parent":null,"max_model_len":32768,"permission":[{"id":"modelperm-ab2b9216bfa44899995a83b6b79e0688","object":"model_permission","created":1738330774,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}


**Python: call APIs**

In [ ]:
from openai import OpenAI
from lmformatenforcer import JsonSchemaParser
from pydantic import BaseModel
from vllm.sampling_params import GuidedDecodingParams
from vllm import SamplingParams

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "https://a07e-35-197-151-225.ngrok-free.app/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

**Direct prompting**

In [ ]:
completion = client.completions.create(model="Qwen/Qwen2.5-1.5B-Instruct",
                                      prompt="San Francisco is a")

print("Chat response:", completion.choices[0].text)


Chat response:  great city for luxury hotels, but we take pride in also being a place that


**Role-based prompting**

In [ ]:
chat_response = client.chat.completions.create(
    model="Qwen/Qwen2.5-1.5B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a joke."},
    ]
)
print("Chat response:", chat_response.choices[0].message.content)

Chat response: Why did the tomato turn red?

Because it saw the salad dressing!


**Structured outputs**

In [ ]:
# Guided decoding by JSON using Pydantic schema
class CarDescription(BaseModel):
    brand: str
    model: str
    car_type: str

class CarList(BaseModel):
    cars: list[CarDescription]

json_schema = CarList.model_json_schema()

prompt = ("Generate a JSON with the brand, model and car_type of"
          "the most 3 iconic cars from the 90's")
completion = client.chat.completions.create(
    model="Qwen/Qwen2.5-1.5B-Instruct",
    messages=[{
        "role": "user",
        "content": prompt,
    }],
    extra_body={"guided_json": json_schema},
)
print(completion.choices[0].message.content)

{
  "cars": [
    {
      "brand": "Honda",
      "model": "Civic",
      "car_type": "sedan"
    },
    {
      "brand": "Ford",
      "model": "Mustang",
      "car_type": "coupe"
    },
    {
      "brand": "Toyota",
      "model": "Corolla",
      "car_type": "sedan"
    }
  ]
}


**Set parameters**

In [ ]:
completion = client.completions.create(model="Qwen/Qwen2.5-1.5B-Instruct",
                                      prompt="San Francisco is a",
                                      max_tokens=100,
                                      temperature=1  # Adjust temperature to control randomness
                                      )

print("Creative Chat response:\n", completion.choices[0].text)


completion = client.completions.create(model="Qwen/Qwen2.5-1.5B-Instruct",
                                      prompt="San Francisco is a",
                                      max_tokens=100,
                                      temperature=0  # Adjust temperature to control randomness
                                      )

print("Uncreative Chat response:\n", completion.choices[0].text)


Creative Chat response:
  big city with 2700, 000 people. The population increases by 12% each year. How many more people will be in San Francisco in 3 years?

To find out how many more people will be in San Francisco in 3 years, we need to calculate the population increase over those years, given the annual increase of 12%. We'll break this down into steps:

1. Calculate the population each year for 3 years.
2. Find
Uncreative Chat response:
  city in the state of California, in the United States. It is the capital and most populous city of the U.S. state of California. It is located on the Pacific coast of the United States, at the mouth of the San Francisco Bay. The city is the largest in the San Francisco Bay Area, the second-largest in California, and the 9th-largest city in the United States. San Francisco is the only major city in the United States to be built on a natural harbor. The


In [ ]:
288.79/8.47/8

4.261953955135773